- [The GDELT Project](https://www.gdeltproject.org/)


- Chen Luo, Feb 18, 2020

## 1. What is GDELT?

- A project supported by Google, monitors the world's broadcast, print, and web news

---

## 2. What makes it special?

- Multilanguage (Over 100)

- A detailed coding system, including people, locations, organizations, themes, emotions

- Open access

- High update frequency (Every 15 mins)

- Historical breadth (some datasets date back to the 19th century)

- [Friendly documents](https://www.gdeltproject.org/data.html#documentation)

---

## 3. Two major datasets

- GDELT Event Database
    - Contains over 300 categories of physical activities over the world
    
    - Nearly 60 attributes are coded for each event

- GDELT Global Knowledge Graph (April 1, 2013 ~ now)
    - Based on each news report, using NER and geocoding algorithms to perform the coding

---

## 4. How to apply GDELT datasets to your network analysis?
- Option 1: GDELT + Gephi
    - [GKG Network Visualizer](http://analysis.gdeltproject.org/module-gkg-network.html)
    
    - Two steps: Enter your keywords, then open your mailbox
    
    - This function is not available now but will come into effect in the next few weeks.
    
    - There are other easy to use [analysis services](http://analysis.gdeltproject.org/)
    
    
- Option 2: Customized data
     - Google BigQuery / Raw data file (`csv` & `tsv`)
     
     - This following demo (very preliminary) creates a countries' co-occurrence (top 1K) network of `global pandemic` (a given theme in GDELT project) through news coverage in the recent week
         - The toolkit includes Google BigQuery ([preview](https://bigquery.cloud.google.com/table/gdelt-bq), but please pay attention to the [quota](https://cloud.google.com/dialogflow/quotas)), Google Cloud Platform, Gephi (geolayout plug-in)

In [1]:
import os
from google.cloud import bigquery
from google.cloud.bigquery.job import QueryJobConfig

# initialize
QueryJobConfig(useLegacySql=False)
# provide the private key of Google Cloud Platform
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './gdelt-a776108ed74c.json'

### 4.1 query for the `edges.csv`

In [3]:
# `gdeltv2.gkg_partitioned` table stores the GKG V2 data (About 11 TB)
# `extra.countrygeolookup` table contains countryname (str), lat (str), long (str), fips (str)
# query_1 is for building the edges table (including source, target, type, weight)
# 'OS' means oceans

query_1 = '''
SELECT
  d.countryname Source, e.countryname Target, "Undirected" Type, ROUND(c.Count/SUM(c.Count) OVER (), 6) Weight
FROM (
  SELECT
    a.countrycode Source, b.countrycode Target, COUNT(*) AS Count
  FROM ( (
      SELECT
        DocumentIdentifier url, REGEXP_EXTRACT(location, r'^.*?#.*?#(.*?)#') countrycode
      FROM
        `gdelt-bq.gdeltv2.gkg_partitioned`, UNNEST(SPLIT(V2Locations, ';')) AS location
      WHERE
        LENGTH(V2Locations) > 3 AND V2Themes LIKE '%HEALTH_PANDEMIC%HEALTH_PANDEMIC%' AND DATE(_PARTITIONTIME) BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY) AND CURRENT_DATE())) a
  JOIN ( (
      SELECT
        DocumentIdentifier url, REGEXP_EXTRACT(location, r'^.*?#.*?#(.*?)#') countrycode
      FROM
        `gdelt-bq.gdeltv2.gkg_partitioned`, UNNEST(SPLIT(V2Locations, ';')) AS location
      WHERE
        LENGTH(V2Locations) > 3 AND V2Themes LIKE '%HEALTH_PANDEMIC%HEALTH_PANDEMIC%' AND DATE(_PARTITIONTIME) BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY) AND CURRENT_DATE())) b
  ON
    a.url=b.url
  WHERE
    a.countrycode < b.countrycode AND a.countrycode != 'OS' AND b.countrycode != 'OS'
  GROUP BY
    1, 2
  ORDER BY
    3 DESC
  LIMIT
    1000) c
JOIN (
  SELECT
    fips, countryname
  FROM
    `gdelt-bq.extra.countrygeolookup`) d
ON
  c.Source = d.fips
JOIN (
  SELECT
    fips, countryname
  FROM
    `gdelt-bq.extra.countrygeolookup`) e
ON
  c.Target = e.fips
ORDER BY
  Count DESC
'''

### 4.2 generate the `nodes.csv`

In [4]:
# query_2 is for building the nodes table (including ID, Label, Lat, Long)
# Lat & Long fields are prepared for the geo-layout
query_2 = '''
SELECT
  Country Id, Country Label, Latitude, Longitude
FROM (
  WITH
    network AS(
    SELECT
      d.countryname Source, d.latitude SourceLatitude, d.longitude SourceLongitude, 
      e.countryname Target, e.latitude TargetLatitude, e.longitude TargetLongitude
    FROM (
      SELECT
        a.countrycode Source, b.countrycode Target, COUNT(*) AS Count
      FROM ( (
          SELECT
            DocumentIdentifier url, REGEXP_EXTRACT(location,r'^.*?#.*?#(.*?)#') countrycode
          FROM
            `gdelt-bq.gdeltv2.gkg_partitioned`, UNNEST(SPLIT(V2Locations, ';')) AS location
          WHERE
            LENGTH(V2Locations) > 3 AND V2Themes LIKE '%HEALTH_PANDEMIC%HEALTH_PANDEMIC%' AND DATE(_PARTITIONTIME) BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY) AND CURRENT_DATE())) a
      JOIN ( (
          SELECT
            DocumentIdentifier url, REGEXP_EXTRACT(location,r'^.*?#.*?#(.*?)#') countrycode
          FROM
            `gdelt-bq.gdeltv2.gkg_partitioned`, UNNEST(SPLIT(V2Locations, ';')) AS location
          WHERE
            LENGTH(V2Locations) > 3 AND V2Themes LIKE '%HEALTH_PANDEMIC%HEALTH_PANDEMIC%' AND DATE(_PARTITIONTIME) BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY) AND CURRENT_DATE())) b
      ON
        a.url=b.url
      WHERE
        a.countrycode < b.countrycode AND a.countrycode != 'OS' AND b.countrycode != 'OS'
      GROUP BY
        1, 2
      ORDER BY
        3 DESC
      LIMIT
        1000) c
    JOIN (
      SELECT
        fips, countryname, latitude, longitude
      FROM
        `gdelt-bq.extra.countrygeolookup`) d
    ON
      c.Source = d.fips
    JOIN (
      SELECT
        fips, countryname, latitude, longitude
      FROM
        `gdelt-bq.extra.countrygeolookup`) e
    ON
      c.Target = e.fips
    ORDER BY
      Count DESC) (
    SELECT
      Source Country, SourceLatitude Latitude, SourceLongitude Longitude
    FROM
      network)
  UNION DISTINCT (
    SELECT
      Target Country, TargetLatitude Latitude, TargetLongitude Longitude
    FROM
      network) )
ORDER BY
  Country
'''

### 4.3 build the query module

In [7]:
class buildNetwork:
    def __init__(self, query, filepath):
        self.client = bigquery.Client()
        self.query = query
        self.filepath = filepath
        self.idx = 0
        
    def query_process(self):
        rows = self.client.query(self.query).result()
        return rows
    
    def save_edges_results(self):
        with open(self.filepath, 'w', encoding='utf-8') as edges:
            edges.write('Source' + ',' + 'Target' + ',' + 'Type' + ',' + 'Weight' + '\n')
            for row in self.query_process():
                self.idx += 1
                edges.write(row['Source'] + ',' + row['Target'] + ',' + row['Type'] + ',' + str(row['Weight']) + '\n')
        return self.idx
    
    def save_nodes_results(self):
        with open(self.filepath, 'w', encoding='utf-8') as nodes:
            nodes.write('ID' + ',' + 'Label' + ',' + 'Latitude' + ',' + 'Longitude' + '\n')
            for row in self.query_process():
                self.idx += 1
                nodes.write(row['Id'] + ',' + row['Label'] + ',' + row['Latitude'] + ',' + str(row['Longitude']) + '\n')
        return self.idx

### 4.4 run the query

In [8]:
%%time
# for the edges
newBuildNetwork = buildNetwork(query_1, './edges_pandemic.csv')
newBuildNetwork.save_edges_results()
print('This query generates %s edges'%(newBuildNetwork.idx))

This query generates 1001 edges
CPU times: user 72.2 ms, sys: 10.5 ms, total: 82.8 ms
Wall time: 7.11 s


In [9]:
%%time
# for the nodes
newBuildNetwork = buildNetwork(query_2, './nodes_pandemic.csv')
newBuildNetwork.save_nodes_results()
print('This query generates %s nodes'%(newBuildNetwork.idx))

This query generates 120 nodes
CPU times: user 52.5 ms, sys: 7.23 ms, total: 59.7 ms
Wall time: 7.96 s


### 4.5 move to Gephi

- The final network

In [13]:
%%html
<img src='./final_network.png' width=1000 height=600>